In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import cv2
from tqdm import tqdm

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential

In [4]:
training_dir = "./CarneDataset/train"

CATEGORIES = ["CLASS_01","CLASS_02","CLASS_03","CLASS_04","CLASS_05","CLASS_06","CLASS_07","CLASS_08"]


training_data= ImageDataGenerator(
    rotation_range=20,  
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train = training_data.flow_from_directory(
    training_dir,
    target_size=(100, 100),
    batch_size=10,
    class_mode='categorical')


Found 1634 images belonging to 8 classes.


In [5]:
test_dir = "./CarneDataset/test"

test_data = ImageDataGenerator(
    rotation_range=20,  
    width_shift_range=0.1,  
    height_shift_range=0.1,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


validation = test_data.flow_from_directory(
    test_dir,
    target_size=(100, 100),
    batch_size=10,
    class_mode='categorical')

Found 810 images belonging to 8 classes.


In [6]:
model = Sequential()

# Primera capa de convolución
model.add(Convolution2D(32, (3, 3), padding ="same", input_shape=(100, 100, 3), activation='relu'))
# Capa de agrupación
model.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda capa de convolución
model.add(Convolution2D(64, (2, 2), padding ="same", activation='relu'))
# Capa de agrupación
model.add(MaxPooling2D(pool_size=(2, 2)))

# Capa de aplanamiento
model.add(Flatten())
# Capa completamente conectada
model.add(Dense(256, activation='relu'))

# Capa de salida
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(learning_rate=0.001),
            metrics=['accuracy'])

In [7]:
summary = model.fit(
    train,
    steps_per_epoch=120, #steps for training
    epochs=10, #epoch for training
    validation_data=validation,
    validation_steps=80 #steps for validate
 )

Epoch 1/10
120/120 [==============================] - 37s 298ms/step - loss: 1.3958 - accuracy: 0.5600 - val_loss: 1.0192 - val_accuracy: 0.6525
Epoch 2/10
120/120 [==============================] - 35s 294ms/step - loss: 0.9671 - accuracy: 0.6709 - val_loss: 0.9128 - val_accuracy: 0.6388
Epoch 3/10
120/120 [==============================] - 43s 361ms/step - loss: 0.7795 - accuracy: 0.7300 - val_loss: 0.5902 - val_accuracy: 0.7912
Epoch 4/10
120/120 [==============================] - 43s 356ms/step - loss: 0.7009 - accuracy: 0.7517 - val_loss: 0.6350 - val_accuracy: 0.7900
Epoch 5/10
120/120 [==============================] - 36s 299ms/step - loss: 0.6438 - accuracy: 0.7708 - val_loss: 0.5541 - val_accuracy: 0.7812
Epoch 6/10
120/120 [==============================] - 36s 297ms/step - loss: 0.5536 - accuracy: 0.8007 - val_loss: 0.5488 - val_accuracy: 0.7775
Epoch 7/10
120/120 [==============================] - 36s 299ms/step - loss: 0.5150 - accuracy: 0.8099 - val_loss: 0.5698 - val_ac

In [52]:
accuracy = summary.history['accuracy']
loss= summary.history['loss']
print(f"Accuracy: {accuracy[-1]:.4f}")
print(f"Loss: {loss[-1]:.4f}")

Accuracy: 0.8174
Loss: 0.4810


In [53]:

#This part is just to save the result of the model and its weight

model_dir = './exam_model/'
if not os.path.exists(model_dir):
  os.mkdir(model_dir)
model.save('./exam_model/model.h5')
model.save_weights('./exam_model/weights.h5')